This jupyter nodebook is for RQ2 of the project to measure what is the minimum required data of SO dataset to achieve acceptable results compared to maximum data size. I start from 10% of the total amount of data to 100%.

# Load data

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
!unzip /content/drive/MyDrive/SE\ project/dataset/EMTK_datasets-v1.0.zip

Archive:  /content/drive/MyDrive/SE project/dataset/EMTK_datasets-v1.0.zip
c200f78878252a3779c76f19be2f0230c38ef00b
   creating: collab-uniba-EMTK_datasets-c200f78/
  inflating: collab-uniba-EMTK_datasets-c200f78/LICENSE  
  inflating: collab-uniba-EMTK_datasets-c200f78/README.md  
   creating: collab-uniba-EMTK_datasets-c200f78/jira/
   creating: collab-uniba-EMTK_datasets-c200f78/jira/emotions/
  inflating: collab-uniba-EMTK_datasets-c200f78/jira/emotions/README.md  
  inflating: collab-uniba-EMTK_datasets-c200f78/jira/emotions/anger.csv  
  inflating: collab-uniba-EMTK_datasets-c200f78/jira/emotions/joy.csv  
  inflating: collab-uniba-EMTK_datasets-c200f78/jira/emotions/love.csv  
  inflating: collab-uniba-EMTK_datasets-c200f78/jira/emotions/sadness.csv  
   creating: collab-uniba-EMTK_datasets-c200f78/so/
   creating: collab-uniba-EMTK_datasets-c200f78/so/emotions/
  inflating: collab-uniba-EMTK_datasets-c200f78/so/emotions/Emotion Annotation Guidelines.pdf  
  inflating: collab-un

In [3]:
import pandas as pd
df = pd.read_excel('/content/collab-uniba-EMTK_datasets-c200f78/so/emotions/Emotions_GoldSandard_andAnnotation.xlsx',sheet_name=None)

In [4]:
df

{'Anger_all':      Group     Set  Unnamed: 2  ... Gold Label Unnamed: 8 Unnamed: 9
 0        A  Second           1  ...        NaN        NaN        NaN
 1        A  Second           2  ...        NaN        NaN        NaN
 2        A  Second           3  ...        NaN        NaN        NaN
 3        A  Second           4  ...        NaN        NaN        NaN
 4        A  Second           5  ...        NaN        NaN        NaN
 ...    ...     ...         ...  ...        ...        ...        ...
 4795     D   Third         496  ...        NaN        NaN        NaN
 4796     D   Third         497  ...        NaN        NaN        NaN
 4797     D   Third         498  ...        NaN        NaN        NaN
 4798     D   Third         499  ...        NaN        NaN        NaN
 4799     D   Third         500  ...        NaN        NaN        NaN
 
 [4800 rows x 10 columns],
 'Fear_all':      Group     Set  Unnamed: 2  ... Gold Label Unnamed: 8 Unnamed: 9
 0        A  Second           1  ...

In [5]:
df["Love_all"]

,Group,Set,Unnamed: 2,Text,rater 1,rater 2,rater 3,Gold Label,Unnamed: 8,Unnamed: 9
0,A,Second,1,SVG transform on text attribute works excellen...,X,NaN,X,LOVE,NaN,NaN
1,A,Second,2,Excellent! This is exactly what I needed. Thanks!,X,x,X,LOVE,NaN,NaN
2,A,Second,3,Have added a modern solution as of May 2014 in...,NaN,NaN,NaN,NaN,NaN,NaN
3,A,Second,4,Have you tried removing 'preload' attribute? (...,NaN,NaN,NaN,NaN,NaN,NaN
4,A,Second,5,"A smarter, entirely C++-way of doing what you ...",NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
4795,D,Third,496,Yes - that feature is extremely useful for wri...,NaN,NaN,x,NaN,NaN,NaN
4796,D,Third,497,"Works great! And you can add ""desc"" after the ...",x,NaN,x,LOVE,NaN,NaN
4797,D,Third,498,"Yeah, I didn't know about the non-greedy thing...",x,NaN,NaN,NaN,NaN,NaN
4798,D,Third,499,Fortunately I'm doing *very* little with Offic...,x,X,x,LOVE,NaN,NaN


In [6]:
text = []
neutral_emotional = []
label_sets = []

for i in range(4800):
  
  label = 0
  label_set = []
  for sheet in ['Love_all', 'Joy_all', 'Anger_all', 'Sadness_all', 'Fear_all', 'Surprise_all']:
    gold_label = df[sheet].loc[i].iat[7]
    if gold_label in ['LOVE', 'JOY', 'ANGER', 'SADNESS', 'FEAR', 'SURPRISE']:
      label = 1
      label_set.append(gold_label)
  if not 'SURPRISE' in label_set:
    neutral_emotional.append(label)
    label_sets.append(label_set)
    text.append(df['Love_all'].loc[i].iat[3])
  

In [7]:
def not_both(label_set,i):
  positive = ['LOVE','JOY']
  negative = ['SADNESS', 'FEAR', 'ANGER']
  is_positive = False
  is_negative = False
  for label in label_set:
    if label in positive:
      is_positive = True
    elif label in negative:
      is_negative = True
  if is_negative and is_positive:
    return False
  else:
    return True

def find_semantic(label_set):
  positive = ['LOVE','JOY']
  negative = ['SADNESS', 'FEAR', 'ANGER']
  if label_set[0] in positive:
    return 1
  return 0

In [8]:
text_semantic = []
semantic = []

for i in range(4755):
  if neutral_emotional[i]:
    if not_both(label_sets[i],i):
      text_semantic.append(text[i])
      semantic.append(find_semantic(label_sets[i]))


In [9]:
text_all = []
three_cat = []

for i in range(4755):
  if neutral_emotional[i]==0:
    text_all.append(text[i])
    three_cat.append('Neutral')
for i in range(len(text_semantic)):
  text_all.append(text_semantic[i])
  if semantic[i]:
    three_cat.append('Positive')
  else:
    three_cat.append('Negative')

In [10]:
len(text_all)

4730

# Model setting

In [11]:
import io
import os
import re
import shutil
import string
import tensorflow as tf
import numpy as np

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, LSTM, Bidirectional
from tensorflow.keras.layers import TextVectorization
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Embedding
from keras.layers import LSTM, GRU
from keras.layers import Flatten, Dense
from keras.callbacks import ModelCheckpoint
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from keras.preprocessing.text import Tokenizer
from keras.preprocessing. sequence import pad_sequences

In [12]:
# define example
data = three_cat
values = array(data)
print(values)
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
print(integer_encoded)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)

['Neutral' 'Neutral' 'Neutral' ... 'Positive' 'Positive' 'Positive']
[1 1 1 ... 2 2 2]
[[0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 ...
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]]


In [13]:
len(text_all)*0.7

3311.0

In [14]:
history_LSTM  = []

# 10 %

In [15]:
rate = 0.1

In [16]:
maxlen = 100 # cut point in the review
training_samples = int(len(text_all)*0.7*rate) # train size
validation_samples = len(text_all)-int(len(text_all)*0.7) # Validates size
max_words = 10000 # Considers only the top 10000 words in the dataset

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(text_all)
sequences = tokenizer.texts_to_sequences(text_all)
word_index = tokenizer.word_index               

data = pad_sequences(sequences, maxlen=maxlen)

labels = np.asarray(onehot_encoded)

indices = np.arange(data.shape[0]) # Splits data into training and validation set, but shuffles is, since samples are ordered: 
# all negatives first, then all positive
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
labels = np.asarray(labels).astype(np.float32)

X_train = data[:training_samples] 
y_train = labels[:training_samples]
X_test = data[training_samples:training_samples+validation_samples] 
y_test = labels[training_samples:training_samples+validation_samples]

In [17]:
# word embedding subsection
embeddings_index = {}

f = open("drive/MyDrive/glove.6B.100d.txt", encoding='utf-8') #added , encoding='utf-8'
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype="float32")
    embeddings_index[word] = coefs
f.close()

print("found %s word vectors." % len (embeddings_index))

found 400000 word vectors.


In [18]:
embedding_dim = 100 # GloVe contains 100-dimensional embedding vectors for 400.000 words

embedding_matrix = np.zeros((max_words, embedding_dim)) # embedding_matrix.shape (10000, 100)
for word, i in word_index.items():
    if i < max_words:
        embedding_vector = embeddings_index.get(word) # embedding_vector.shape (100,)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector # Words not found in the mebedding index will all be zeros

In [19]:
d= 80

# built the model
model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length = maxlen))
model.add(LSTM(d, dropout = 0.3))
model.add(Dense(3, activation = "softmax"))
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False

# compile
model.compile(optimizer = "rmsprop",
            loss = "categorical_crossentropy",
            metrics = ["acc"])

In [20]:
# fit the model
mcp_save = ModelCheckpoint('model_best_1.hdf5', save_best_only=True, monitor='val_loss', mode='min')
history = model.fit(X_train, y_train, epochs=15, batch_size=8, validation_data=(X_test,y_test), callbacks=[mcp_save])

Epoch 1/15
42/42 [==============================] - 9s 74ms/step - loss: 1.0660 - acc: 0.4804 - val_loss: 1.0336 - val_acc: 0.4489
Epoch 2/15
42/42 [==============================] - 2s 60ms/step - loss: 0.9734 - acc: 0.5438 - val_loss: 0.9220 - val_acc: 0.6075
Epoch 3/15
42/42 [==============================] - 3s 61ms/step - loss: 0.8705 - acc: 0.6556 - val_loss: 0.8513 - val_acc: 0.6385
Epoch 4/15
42/42 [==============================] - 2s 59ms/step - loss: 0.7772 - acc: 0.6526 - val_loss: 0.9527 - val_acc: 0.5384
Epoch 5/15
42/42 [==============================] - 2s 59ms/step - loss: 0.7252 - acc: 0.6828 - val_loss: 0.7902 - val_acc: 0.6540
Epoch 6/15
42/42 [==============================] - 2s 59ms/step - loss: 0.6429 - acc: 0.7341 - val_loss: 0.7643 - val_acc: 0.6822
Epoch 7/15
42/42 [==============================] - 4s 85ms/step - loss: 0.6427 - acc: 0.7432 - val_loss: 0.7453 - val_acc: 0.6906
Epoch 8/15
42/42 [==============================] - 2s 60ms/step - loss: 0.5940 - a

In [21]:
from keras.models import load_model
model = load_model('/content/model_best_1.hdf5')

In [22]:
y_pred = np.argmax(model.predict(X_test), axis=1)
y_test_1 = np.argmax(y_test, axis=1)

In [23]:
from sklearn.metrics import classification_report

accu = classification_report(y_test_1, y_pred, output_dict = True)['accuracy']
history_LSTM.append(accu)

# 20 %

In [24]:
rate = 0.2

In [25]:
maxlen = 100 # cut point in the review
training_samples = int(len(text_all)*0.7*rate) # train size
validation_samples = len(text_all)-int(len(text_all)*0.7) # Validates size
max_words = 10000 # Considers only the top 10000 words in the dataset

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(text_all)
sequences = tokenizer.texts_to_sequences(text_all)
word_index = tokenizer.word_index               

data = pad_sequences(sequences, maxlen=maxlen)

labels = np.asarray(onehot_encoded)

indices = np.arange(data.shape[0]) # Splits data into training and validation set, but shuffles is, since samples are ordered: 
# all negatives first, then all positive
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
labels = np.asarray(labels).astype(np.float32)

X_train = data[:training_samples] 
y_train = labels[:training_samples]
X_test = data[training_samples:training_samples+validation_samples] 
y_test = labels[training_samples:training_samples+validation_samples]

In [26]:
# word embedding subsection
embeddings_index = {}

f = open("drive/MyDrive/glove.6B.100d.txt", encoding='utf-8') #added , encoding='utf-8'
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype="float32")
    embeddings_index[word] = coefs
f.close()

print("found %s word vectors." % len (embeddings_index))

found 400000 word vectors.


In [27]:
embedding_dim = 100 # GloVe contains 100-dimensional embedding vectors for 400.000 words

embedding_matrix = np.zeros((max_words, embedding_dim)) # embedding_matrix.shape (10000, 100)
for word, i in word_index.items():
    if i < max_words:
        embedding_vector = embeddings_index.get(word) # embedding_vector.shape (100,)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector # Words not found in the mebedding index will all be zeros

In [28]:
d= 80

# built the model
model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length = maxlen))
model.add(LSTM(d, dropout = 0.3))
model.add(Dense(3, activation = "softmax"))
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False

# compile
model.compile(optimizer = "rmsprop",
            loss = "categorical_crossentropy",
            metrics = ["acc"])

In [29]:
# fit the model
mcp_save = ModelCheckpoint('model_best_2.hdf5', save_best_only=True, monitor='val_loss', mode='min')
history = model.fit(X_train, y_train, epochs=15, batch_size=8, validation_data=(X_test,y_test), callbacks=[mcp_save])

Epoch 1/15
83/83 [==============================] - 7s 61ms/step - loss: 0.9925 - acc: 0.5136 - val_loss: 1.0158 - val_acc: 0.5159
Epoch 2/15
83/83 [==============================] - 3s 41ms/step - loss: 0.8413 - acc: 0.6344 - val_loss: 0.8213 - val_acc: 0.6342
Epoch 3/15
83/83 [==============================] - 4s 54ms/step - loss: 0.7573 - acc: 0.6873 - val_loss: 0.7894 - val_acc: 0.6638
Epoch 4/15
83/83 [==============================] - 3s 41ms/step - loss: 0.6988 - acc: 0.7205 - val_loss: 0.7484 - val_acc: 0.6681
Epoch 5/15
83/83 [==============================] - 3s 41ms/step - loss: 0.6225 - acc: 0.7613 - val_loss: 0.8297 - val_acc: 0.6610
Epoch 6/15
83/83 [==============================] - 3s 41ms/step - loss: 0.6006 - acc: 0.7613 - val_loss: 0.9859 - val_acc: 0.5934
Epoch 7/15
83/83 [==============================] - 3s 41ms/step - loss: 0.5812 - acc: 0.7764 - val_loss: 0.7014 - val_acc: 0.7005
Epoch 8/15
83/83 [==============================] - 4s 54ms/step - loss: 0.5310 - a

In [30]:
from keras.models import load_model
model = load_model('/content/model_best_2.hdf5')

In [31]:
y_pred = np.argmax(model.predict(X_test), axis=1)
y_test_1 = np.argmax(y_test, axis=1)

In [32]:
from sklearn.metrics import classification_report

accu = classification_report(y_test_1, y_pred, output_dict = True)['accuracy']
history_LSTM.append(accu)

# 30 %

In [33]:
rate = 0.3

In [34]:
maxlen = 100 # cut point in the review
training_samples = int(len(text_all)*0.7*rate) # train size
validation_samples = len(text_all)-int(len(text_all)*0.7) # Validates size
max_words = 10000 # Considers only the top 10000 words in the dataset

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(text_all)
sequences = tokenizer.texts_to_sequences(text_all)
word_index = tokenizer.word_index               

data = pad_sequences(sequences, maxlen=maxlen)

labels = np.asarray(onehot_encoded)

indices = np.arange(data.shape[0]) # Splits data into training and validation set, but shuffles is, since samples are ordered: 
# all negatives first, then all positive
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
labels = np.asarray(labels).astype(np.float32)

X_train = data[:training_samples] 
y_train = labels[:training_samples]
X_test = data[training_samples:training_samples+validation_samples] 
y_test = labels[training_samples:training_samples+validation_samples]

In [35]:
# word embedding subsection
embeddings_index = {}

f = open("drive/MyDrive/glove.6B.100d.txt", encoding='utf-8') #added , encoding='utf-8'
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype="float32")
    embeddings_index[word] = coefs
f.close()

print("found %s word vectors." % len (embeddings_index))

found 400000 word vectors.


In [36]:
embedding_dim = 100 # GloVe contains 100-dimensional embedding vectors for 400.000 words

embedding_matrix = np.zeros((max_words, embedding_dim)) # embedding_matrix.shape (10000, 100)
for word, i in word_index.items():
    if i < max_words:
        embedding_vector = embeddings_index.get(word) # embedding_vector.shape (100,)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector # Words not found in the mebedding index will all be zeros

In [37]:
d= 80

# built the model
model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length = maxlen))
model.add(LSTM(d, dropout = 0.3))
model.add(Dense(3, activation = "softmax"))
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False

# compile
model.compile(optimizer = "rmsprop",
            loss = "categorical_crossentropy",
            metrics = ["acc"])

In [38]:
# fit the model
mcp_save = ModelCheckpoint('model_best_3.hdf5', save_best_only=True, monitor='val_loss', mode='min')
history = model.fit(X_train, y_train, epochs=15, batch_size=8, validation_data=(X_test,y_test), callbacks=[mcp_save])

Epoch 1/15
125/125 [==============================] - 9s 51ms/step - loss: 0.9738 - acc: 0.5438 - val_loss: 1.0231 - val_acc: 0.5292
Epoch 2/15
125/125 [==============================] - 4s 35ms/step - loss: 0.7983 - acc: 0.6506 - val_loss: 1.0310 - val_acc: 0.5574
Epoch 3/15
125/125 [==============================] - 4s 35ms/step - loss: 0.7336 - acc: 0.6757 - val_loss: 0.7094 - val_acc: 0.6991
Epoch 4/15
125/125 [==============================] - 5s 44ms/step - loss: 0.6659 - acc: 0.7311 - val_loss: 0.6659 - val_acc: 0.7174
Epoch 5/15
125/125 [==============================] - 4s 35ms/step - loss: 0.6315 - acc: 0.7362 - val_loss: 0.7718 - val_acc: 0.6582
Epoch 6/15
125/125 [==============================] - 4s 35ms/step - loss: 0.5895 - acc: 0.7644 - val_loss: 0.6085 - val_acc: 0.7590
Epoch 7/15
125/125 [==============================] - 4s 35ms/step - loss: 0.5650 - acc: 0.7754 - val_loss: 0.5836 - val_acc: 0.7653
Epoch 8/15
125/125 [==============================] - 4s 35ms/step - 

In [39]:
from keras.models import load_model
model = load_model('/content/model_best_3.hdf5')

In [40]:
y_pred = np.argmax(model.predict(X_test), axis=1)
y_test_1 = np.argmax(y_test, axis=1)

In [41]:
from sklearn.metrics import classification_report

accu = classification_report(y_test_1, y_pred, output_dict = True)['accuracy']
history_LSTM.append(accu)

# 40 %

In [42]:
rate = 0.4

In [43]:
maxlen = 100 # cut point in the review
training_samples = int(len(text_all)*0.7*rate) # train size
validation_samples = len(text_all)-int(len(text_all)*0.7) # Validates size
max_words = 10000 # Considers only the top 10000 words in the dataset

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(text_all)
sequences = tokenizer.texts_to_sequences(text_all)
word_index = tokenizer.word_index               

data = pad_sequences(sequences, maxlen=maxlen)

labels = np.asarray(onehot_encoded)

indices = np.arange(data.shape[0]) # Splits data into training and validation set, but shuffles is, since samples are ordered: 
# all negatives first, then all positive
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
labels = np.asarray(labels).astype(np.float32)

X_train = data[:training_samples] 
y_train = labels[:training_samples]
X_test = data[training_samples:training_samples+validation_samples] 
y_test = labels[training_samples:training_samples+validation_samples]

In [44]:
# word embedding subsection
embeddings_index = {}

f = open("drive/MyDrive/glove.6B.100d.txt", encoding='utf-8') #added , encoding='utf-8'
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype="float32")
    embeddings_index[word] = coefs
f.close()

print("found %s word vectors." % len (embeddings_index))

found 400000 word vectors.


In [45]:
embedding_dim = 100 # GloVe contains 100-dimensional embedding vectors for 400.000 words

embedding_matrix = np.zeros((max_words, embedding_dim)) # embedding_matrix.shape (10000, 100)
for word, i in word_index.items():
    if i < max_words:
        embedding_vector = embeddings_index.get(word) # embedding_vector.shape (100,)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector # Words not found in the mebedding index will all be zeros

In [46]:
d= 80

# built the model
model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length = maxlen))
model.add(LSTM(d, dropout = 0.3))
model.add(Dense(3, activation = "softmax"))
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False

# compile
model.compile(optimizer = "rmsprop",
            loss = "categorical_crossentropy",
            metrics = ["acc"])

In [47]:
# fit the model
mcp_save = ModelCheckpoint('model_best_4.hdf5', save_best_only=True, monitor='val_loss', mode='min')
history = model.fit(X_train, y_train, epochs=15, batch_size=8, validation_data=(X_test,y_test), callbacks=[mcp_save])

Epoch 1/15
166/166 [==============================] - 9s 42ms/step - loss: 0.9364 - acc: 0.5718 - val_loss: 0.7744 - val_acc: 0.6709
Epoch 2/15
166/166 [==============================] - 5s 31ms/step - loss: 0.7902 - acc: 0.6480 - val_loss: 0.6938 - val_acc: 0.7019
Epoch 3/15
166/166 [==============================] - 5s 32ms/step - loss: 0.7210 - acc: 0.7024 - val_loss: 0.6437 - val_acc: 0.7329
Epoch 4/15
166/166 [==============================] - 6s 38ms/step - loss: 0.6476 - acc: 0.7379 - val_loss: 1.0423 - val_acc: 0.6547
Epoch 5/15
166/166 [==============================] - 5s 32ms/step - loss: 0.5877 - acc: 0.7628 - val_loss: 0.6923 - val_acc: 0.7259
Epoch 6/15
166/166 [==============================] - 6s 38ms/step - loss: 0.5518 - acc: 0.7817 - val_loss: 0.5810 - val_acc: 0.7766
Epoch 7/15
166/166 [==============================] - 6s 38ms/step - loss: 0.5039 - acc: 0.7998 - val_loss: 0.5726 - val_acc: 0.7646
Epoch 8/15
166/166 [==============================] - 5s 32ms/step - 

In [48]:
from keras.models import load_model
model = load_model('/content/model_best_4.hdf5')

In [49]:
y_pred = np.argmax(model.predict(X_test), axis=1)
y_test_1 = np.argmax(y_test, axis=1)

In [50]:
from sklearn.metrics import classification_report

accu = classification_report(y_test_1, y_pred, output_dict = True)['accuracy']
history_LSTM.append(accu)

# 50 %

In [51]:
rate = 0.5

In [52]:
maxlen = 100 # cut point in the review
training_samples = int(len(text_all)*0.7*rate) # train size
validation_samples = len(text_all)-int(len(text_all)*0.7) # Validates size
max_words = 10000 # Considers only the top 10000 words in the dataset

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(text_all)
sequences = tokenizer.texts_to_sequences(text_all)
word_index = tokenizer.word_index               

data = pad_sequences(sequences, maxlen=maxlen)

labels = np.asarray(onehot_encoded)

indices = np.arange(data.shape[0]) # Splits data into training and validation set, but shuffles is, since samples are ordered: 
# all negatives first, then all positive
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
labels = np.asarray(labels).astype(np.float32)

X_train = data[:training_samples] 
y_train = labels[:training_samples]
X_test = data[training_samples:training_samples+validation_samples] 
y_test = labels[training_samples:training_samples+validation_samples]

In [53]:
# word embedding subsection
embeddings_index = {}

f = open("drive/MyDrive/glove.6B.100d.txt", encoding='utf-8') #added , encoding='utf-8'
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype="float32")
    embeddings_index[word] = coefs
f.close()

print("found %s word vectors." % len (embeddings_index))

found 400000 word vectors.


In [54]:
embedding_dim = 100 # GloVe contains 100-dimensional embedding vectors for 400.000 words

embedding_matrix = np.zeros((max_words, embedding_dim)) # embedding_matrix.shape (10000, 100)
for word, i in word_index.items():
    if i < max_words:
        embedding_vector = embeddings_index.get(word) # embedding_vector.shape (100,)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector # Words not found in the mebedding index will all be zeros

In [55]:
d= 80

# built the model
model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length = maxlen))
model.add(LSTM(d, dropout = 0.3))
model.add(Dense(3, activation = "softmax"))
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False

# compile
model.compile(optimizer = "rmsprop",
            loss = "categorical_crossentropy",
            metrics = ["acc"])

In [56]:
# fit the model
mcp_save = ModelCheckpoint('model_best_5.hdf5', save_best_only=True, monitor='val_loss', mode='min')
history = model.fit(X_train, y_train, epochs=15, batch_size=8, validation_data=(X_test,y_test), callbacks=[mcp_save])

Epoch 1/15
207/207 [==============================] - 9s 33ms/step - loss: 0.9326 - acc: 0.5801 - val_loss: 0.8050 - val_acc: 0.6321
Epoch 2/15
207/207 [==============================] - 6s 30ms/step - loss: 0.7582 - acc: 0.6767 - val_loss: 0.7448 - val_acc: 0.6815
Epoch 3/15
207/207 [==============================] - 6s 30ms/step - loss: 0.7005 - acc: 0.7172 - val_loss: 0.6706 - val_acc: 0.7097
Epoch 4/15
207/207 [==============================] - 7s 35ms/step - loss: 0.6284 - acc: 0.7444 - val_loss: 0.6818 - val_acc: 0.6977
Epoch 5/15
207/207 [==============================] - 6s 30ms/step - loss: 0.5703 - acc: 0.7668 - val_loss: 0.6594 - val_acc: 0.7555
Epoch 6/15
207/207 [==============================] - 6s 30ms/step - loss: 0.5394 - acc: 0.7861 - val_loss: 0.5843 - val_acc: 0.7689
Epoch 7/15
207/207 [==============================] - 7s 35ms/step - loss: 0.5060 - acc: 0.7903 - val_loss: 0.5765 - val_acc: 0.7583
Epoch 8/15
207/207 [==============================] - 6s 30ms/step - 

In [57]:
from keras.models import load_model
model = load_model('/content/model_best_5.hdf5')

In [58]:
y_pred = np.argmax(model.predict(X_test), axis=1)
y_test_1 = np.argmax(y_test, axis=1)

In [59]:
from sklearn.metrics import classification_report

accu = classification_report(y_test_1, y_pred, output_dict = True)['accuracy']
history_LSTM.append(accu)

# 60 %

In [60]:
rate = 0.6

In [61]:
maxlen = 100 # cut point in the review
training_samples = int(len(text_all)*0.7*rate) # train size
validation_samples = len(text_all)-int(len(text_all)*0.7) # Validates size
max_words = 10000 # Considers only the top 10000 words in the dataset

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(text_all)
sequences = tokenizer.texts_to_sequences(text_all)
word_index = tokenizer.word_index               

data = pad_sequences(sequences, maxlen=maxlen)

labels = np.asarray(onehot_encoded)

indices = np.arange(data.shape[0]) # Splits data into training and validation set, but shuffles is, since samples are ordered: 
# all negatives first, then all positive
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
labels = np.asarray(labels).astype(np.float32)

X_train = data[:training_samples] 
y_train = labels[:training_samples]
X_test = data[training_samples:training_samples+validation_samples] 
y_test = labels[training_samples:training_samples+validation_samples]

In [62]:
# word embedding subsection
embeddings_index = {}

f = open("drive/MyDrive/glove.6B.100d.txt", encoding='utf-8') #added , encoding='utf-8'
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype="float32")
    embeddings_index[word] = coefs
f.close()

print("found %s word vectors." % len (embeddings_index))

found 400000 word vectors.


In [63]:
embedding_dim = 100 # GloVe contains 100-dimensional embedding vectors for 400.000 words

embedding_matrix = np.zeros((max_words, embedding_dim)) # embedding_matrix.shape (10000, 100)
for word, i in word_index.items():
    if i < max_words:
        embedding_vector = embeddings_index.get(word) # embedding_vector.shape (100,)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector # Words not found in the mebedding index will all be zeros

In [64]:
d= 80

# built the model
model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length = maxlen))
model.add(LSTM(d, dropout = 0.3))
model.add(Dense(3, activation = "softmax"))
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False

# compile
model.compile(optimizer = "rmsprop",
            loss = "categorical_crossentropy",
            metrics = ["acc"])

In [65]:
# fit the model
mcp_save = ModelCheckpoint('model_best_6.hdf5', save_best_only=True, monitor='val_loss', mode='min')
history = model.fit(X_train, y_train, epochs=15, batch_size=8, validation_data=(X_test,y_test), callbacks=[mcp_save])

Epoch 1/15
249/249 [==============================] - 10s 31ms/step - loss: 0.9076 - acc: 0.5947 - val_loss: 0.8486 - val_acc: 0.6096
Epoch 2/15
249/249 [==============================] - 7s 28ms/step - loss: 0.7446 - acc: 0.6883 - val_loss: 0.9144 - val_acc: 0.6441
Epoch 3/15
249/249 [==============================] - 7s 28ms/step - loss: 0.6748 - acc: 0.7231 - val_loss: 0.6581 - val_acc: 0.7294
Epoch 4/15
249/249 [==============================] - 8s 33ms/step - loss: 0.5892 - acc: 0.7608 - val_loss: 0.6590 - val_acc: 0.7174
Epoch 5/15
249/249 [==============================] - 8s 33ms/step - loss: 0.5380 - acc: 0.7875 - val_loss: 0.6341 - val_acc: 0.7442
Epoch 6/15
249/249 [==============================] - 7s 28ms/step - loss: 0.5030 - acc: 0.8051 - val_loss: 0.5610 - val_acc: 0.7815
Epoch 7/15
249/249 [==============================] - 7s 28ms/step - loss: 0.4831 - acc: 0.8082 - val_loss: 0.6856 - val_acc: 0.7414
Epoch 8/15
249/249 [==============================] - 8s 33ms/step -

In [66]:
from keras.models import load_model
model = load_model('/content/model_best_6.hdf5')

In [67]:
y_pred = np.argmax(model.predict(X_test), axis=1)
y_test_1 = np.argmax(y_test, axis=1)

In [68]:
from sklearn.metrics import classification_report

accu = classification_report(y_test_1, y_pred, output_dict = True)['accuracy']
history_LSTM.append(accu)

# 70 %

In [69]:
rate = 0.7

In [70]:
maxlen = 100 # cut point in the review
training_samples = int(len(text_all)*0.7*rate) # train size
validation_samples = len(text_all)-int(len(text_all)*0.7) # Validates size
max_words = 10000 # Considers only the top 10000 words in the dataset

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(text_all)
sequences = tokenizer.texts_to_sequences(text_all)
word_index = tokenizer.word_index               

data = pad_sequences(sequences, maxlen=maxlen)

labels = np.asarray(onehot_encoded)

indices = np.arange(data.shape[0]) # Splits data into training and validation set, but shuffles is, since samples are ordered: 
# all negatives first, then all positive
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
labels = np.asarray(labels).astype(np.float32)

X_train = data[:training_samples] 
y_train = labels[:training_samples]
X_test = data[training_samples:training_samples+validation_samples] 
y_test = labels[training_samples:training_samples+validation_samples]

In [71]:
# word embedding subsection
embeddings_index = {}

f = open("drive/MyDrive/glove.6B.100d.txt", encoding='utf-8') #added , encoding='utf-8'
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype="float32")
    embeddings_index[word] = coefs
f.close()

print("found %s word vectors." % len (embeddings_index))

found 400000 word vectors.


In [72]:
embedding_dim = 100 # GloVe contains 100-dimensional embedding vectors for 400.000 words

embedding_matrix = np.zeros((max_words, embedding_dim)) # embedding_matrix.shape (10000, 100)
for word, i in word_index.items():
    if i < max_words:
        embedding_vector = embeddings_index.get(word) # embedding_vector.shape (100,)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector # Words not found in the mebedding index will all be zeros

In [73]:
d= 80

# built the model
model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length = maxlen))
model.add(LSTM(d, dropout = 0.3))
model.add(Dense(3, activation = "softmax"))
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False

# compile
model.compile(optimizer = "rmsprop",
            loss = "categorical_crossentropy",
            metrics = ["acc"])

In [74]:
# fit the model
mcp_save = ModelCheckpoint('model_best_7.hdf5', save_best_only=True, monitor='val_loss', mode='min')
history = model.fit(X_train, y_train, epochs=15, batch_size=8, validation_data=(X_test,y_test), callbacks=[mcp_save])

Epoch 1/15
290/290 [==============================] - 11s 29ms/step - loss: 0.9100 - acc: 0.5874 - val_loss: 0.7469 - val_acc: 0.6744
Epoch 2/15
290/290 [==============================] - 8s 27ms/step - loss: 0.7476 - acc: 0.6867 - val_loss: 0.7583 - val_acc: 0.7082
Epoch 3/15
290/290 [==============================] - 8s 27ms/step - loss: 0.6742 - acc: 0.7315 - val_loss: 0.6208 - val_acc: 0.7463
Epoch 4/15
290/290 [==============================] - 8s 27ms/step - loss: 0.5875 - acc: 0.7678 - val_loss: 0.5505 - val_acc: 0.7829
Epoch 5/15
290/290 [==============================] - 8s 27ms/step - loss: 0.5491 - acc: 0.7902 - val_loss: 0.5369 - val_acc: 0.7984
Epoch 6/15
290/290 [==============================] - 8s 27ms/step - loss: 0.5098 - acc: 0.8041 - val_loss: 0.5333 - val_acc: 0.7844
Epoch 7/15
290/290 [==============================] - 9s 31ms/step - loss: 0.4810 - acc: 0.8144 - val_loss: 0.5596 - val_acc: 0.7674
Epoch 8/15
290/290 [==============================] - 8s 27ms/step -

In [75]:
from keras.models import load_model
model = load_model('/content/model_best_7.hdf5')

In [76]:
y_pred = np.argmax(model.predict(X_test), axis=1)
y_test_1 = np.argmax(y_test, axis=1)

In [77]:
from sklearn.metrics import classification_report

accu = classification_report(y_test_1, y_pred, output_dict = True)['accuracy']
history_LSTM.append(accu)

# 80 %

In [78]:
rate = 0.8

In [79]:
maxlen = 100 # cut point in the review
training_samples = int(len(text_all)*0.7*rate) # train size
validation_samples = len(text_all)-int(len(text_all)*0.7) # Validates size
max_words = 10000 # Considers only the top 10000 words in the dataset

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(text_all)
sequences = tokenizer.texts_to_sequences(text_all)
word_index = tokenizer.word_index               

data = pad_sequences(sequences, maxlen=maxlen)

labels = np.asarray(onehot_encoded)

indices = np.arange(data.shape[0]) # Splits data into training and validation set, but shuffles is, since samples are ordered: 
# all negatives first, then all positive
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
labels = np.asarray(labels).astype(np.float32)

X_train = data[:training_samples] 
y_train = labels[:training_samples]
X_test = data[training_samples:training_samples+validation_samples] 
y_test = labels[training_samples:training_samples+validation_samples]

In [80]:
# word embedding subsection
embeddings_index = {}

f = open("drive/MyDrive/glove.6B.100d.txt", encoding='utf-8') #added , encoding='utf-8'
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype="float32")
    embeddings_index[word] = coefs
f.close()

print("found %s word vectors." % len (embeddings_index))

found 400000 word vectors.


In [81]:
embedding_dim = 100 # GloVe contains 100-dimensional embedding vectors for 400.000 words

embedding_matrix = np.zeros((max_words, embedding_dim)) # embedding_matrix.shape (10000, 100)
for word, i in word_index.items():
    if i < max_words:
        embedding_vector = embeddings_index.get(word) # embedding_vector.shape (100,)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector # Words not found in the mebedding index will all be zeros

In [82]:
d= 80

# built the model
model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length = maxlen))
model.add(LSTM(d, dropout = 0.3))
model.add(Dense(3, activation = "softmax"))
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False

# compile
model.compile(optimizer = "rmsprop",
            loss = "categorical_crossentropy",
            metrics = ["acc"])

In [83]:
# fit the model
mcp_save = ModelCheckpoint('model_best_8.hdf5', save_best_only=True, monitor='val_loss', mode='min')
history = model.fit(X_train, y_train, epochs=15, batch_size=8, validation_data=(X_test,y_test), callbacks=[mcp_save])

Epoch 1/15
331/331 [==============================] - 12s 31ms/step - loss: 0.8886 - acc: 0.5940 - val_loss: 0.8257 - val_acc: 0.6392
Epoch 2/15
331/331 [==============================] - 11s 33ms/step - loss: 0.7128 - acc: 0.7002 - val_loss: 0.7150 - val_acc: 0.7223
Epoch 3/15
331/331 [==============================] - 10s 30ms/step - loss: 0.6212 - acc: 0.7455 - val_loss: 0.5907 - val_acc: 0.7583
Epoch 4/15
331/331 [==============================] - 11s 33ms/step - loss: 0.5586 - acc: 0.7764 - val_loss: 0.5297 - val_acc: 0.7942
Epoch 5/15
331/331 [==============================] - 10s 29ms/step - loss: 0.5210 - acc: 0.8032 - val_loss: 0.5321 - val_acc: 0.7970
Epoch 6/15
331/331 [==============================] - 10s 29ms/step - loss: 0.4930 - acc: 0.8089 - val_loss: 0.5255 - val_acc: 0.8013
Epoch 7/15
331/331 [==============================] - 11s 33ms/step - loss: 0.4698 - acc: 0.8214 - val_loss: 0.5239 - val_acc: 0.8069
Epoch 8/15
331/331 [==============================] - 10s 30ms

In [84]:
from keras.models import load_model
model = load_model('/content/model_best_8.hdf5')

In [85]:
y_pred = np.argmax(model.predict(X_test), axis=1)
y_test_1 = np.argmax(y_test, axis=1)

In [86]:
from sklearn.metrics import classification_report

accu = classification_report(y_test_1, y_pred, output_dict = True)['accuracy']
history_LSTM.append(accu)

# 90 %

In [87]:
rate = 0.9

In [88]:
maxlen = 100 # cut point in the review
training_samples = int(len(text_all)*0.7*rate) # train size
validation_samples = len(text_all)-int(len(text_all)*0.7) # Validates size
max_words = 10000 # Considers only the top 10000 words in the dataset

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(text_all)
sequences = tokenizer.texts_to_sequences(text_all)
word_index = tokenizer.word_index               

data = pad_sequences(sequences, maxlen=maxlen)

labels = np.asarray(onehot_encoded)

indices = np.arange(data.shape[0]) # Splits data into training and validation set, but shuffles is, since samples are ordered: 
# all negatives first, then all positive
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
labels = np.asarray(labels).astype(np.float32)

X_train = data[:training_samples] 
y_train = labels[:training_samples]
X_test = data[training_samples:training_samples+validation_samples] 
y_test = labels[training_samples:training_samples+validation_samples]

In [89]:
# word embedding subsection
embeddings_index = {}

f = open("drive/MyDrive/glove.6B.100d.txt", encoding='utf-8') #added , encoding='utf-8'
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype="float32")
    embeddings_index[word] = coefs
f.close()

print("found %s word vectors." % len (embeddings_index))

found 400000 word vectors.


In [90]:
embedding_dim = 100 # GloVe contains 100-dimensional embedding vectors for 400.000 words

embedding_matrix = np.zeros((max_words, embedding_dim)) # embedding_matrix.shape (10000, 100)
for word, i in word_index.items():
    if i < max_words:
        embedding_vector = embeddings_index.get(word) # embedding_vector.shape (100,)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector # Words not found in the mebedding index will all be zeros

In [91]:
d= 80

# built the model
model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length = maxlen))
model.add(LSTM(d, dropout = 0.3))
model.add(Dense(3, activation = "softmax"))
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False

# compile
model.compile(optimizer = "rmsprop",
            loss = "categorical_crossentropy",
            metrics = ["acc"])

In [92]:
# fit the model
mcp_save = ModelCheckpoint('model_best_9.hdf5', save_best_only=True, monitor='val_loss', mode='min')
history = model.fit(X_train, y_train, epochs=15, batch_size=8, validation_data=(X_test,y_test), callbacks=[mcp_save])

Epoch 1/15
373/373 [==============================] - 15s 32ms/step - loss: 0.8721 - acc: 0.6052 - val_loss: 0.7351 - val_acc: 0.6836
Epoch 2/15
373/373 [==============================] - 10s 26ms/step - loss: 0.7058 - acc: 0.7036 - val_loss: 0.6378 - val_acc: 0.7259
Epoch 3/15
373/373 [==============================] - 10s 26ms/step - loss: 0.6124 - acc: 0.7486 - val_loss: 0.5992 - val_acc: 0.7576
Epoch 4/15
373/373 [==============================] - 10s 26ms/step - loss: 0.5583 - acc: 0.7707 - val_loss: 0.5704 - val_acc: 0.7745
Epoch 5/15
373/373 [==============================] - 11s 29ms/step - loss: 0.5232 - acc: 0.7949 - val_loss: 0.6245 - val_acc: 0.7773
Epoch 6/15
373/373 [==============================] - 10s 26ms/step - loss: 0.4737 - acc: 0.8184 - val_loss: 0.5695 - val_acc: 0.7935
Epoch 7/15
373/373 [==============================] - 10s 26ms/step - loss: 0.4612 - acc: 0.8228 - val_loss: 0.5056 - val_acc: 0.8259
Epoch 8/15
373/373 [==============================] - 11s 29ms

In [93]:
from keras.models import load_model
model = load_model('/content/model_best_9.hdf5')

In [94]:
y_pred = np.argmax(model.predict(X_test), axis=1)
y_test_1 = np.argmax(y_test, axis=1)

In [95]:
from sklearn.metrics import classification_report

accu = classification_report(y_test_1, y_pred, output_dict = True)['accuracy']
history_LSTM.append(accu)

# 100 %

In [96]:
rate = 1

In [97]:
maxlen = 100 # cut point in the review
training_samples = int(len(text_all)*0.7*rate) # train size
validation_samples = len(text_all)-int(len(text_all)*0.7) # Validates size
max_words = 10000 # Considers only the top 10000 words in the dataset

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(text_all)
sequences = tokenizer.texts_to_sequences(text_all)
word_index = tokenizer.word_index               

data = pad_sequences(sequences, maxlen=maxlen)

labels = np.asarray(onehot_encoded)

indices = np.arange(data.shape[0]) # Splits data into training and validation set, but shuffles is, since samples are ordered: 
# all negatives first, then all positive
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
labels = np.asarray(labels).astype(np.float32)

X_train = data[:training_samples] 
y_train = labels[:training_samples]
X_test = data[training_samples:training_samples+validation_samples] 
y_test = labels[training_samples:training_samples+validation_samples]

In [98]:
# word embedding subsection
embeddings_index = {}

f = open("drive/MyDrive/glove.6B.100d.txt", encoding='utf-8') #added , encoding='utf-8'
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype="float32")
    embeddings_index[word] = coefs
f.close()

print("found %s word vectors." % len (embeddings_index))

found 400000 word vectors.


In [99]:
embedding_dim = 100 # GloVe contains 100-dimensional embedding vectors for 400.000 words

embedding_matrix = np.zeros((max_words, embedding_dim)) # embedding_matrix.shape (10000, 100)
for word, i in word_index.items():
    if i < max_words:
        embedding_vector = embeddings_index.get(word) # embedding_vector.shape (100,)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector # Words not found in the mebedding index will all be zeros

In [100]:
d= 80

# built the model
model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length = maxlen))
model.add(LSTM(d, dropout = 0.3))
model.add(Dense(3, activation = "softmax"))
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False

# compile
model.compile(optimizer = "rmsprop",
            loss = "categorical_crossentropy",
            metrics = ["acc"])

In [101]:
# fit the model
mcp_save = ModelCheckpoint('model_best_100.hdf5', save_best_only=True, monitor='val_loss', mode='min')
history = model.fit(X_train, y_train, epochs=15, batch_size=8, validation_data=(X_test,y_test), callbacks=[mcp_save])

Epoch 1/15
414/414 [==============================] - 14s 27ms/step - loss: 0.8592 - acc: 0.6249 - val_loss: 0.6912 - val_acc: 0.7153
Epoch 2/15
414/414 [==============================] - 11s 26ms/step - loss: 0.7041 - acc: 0.7101 - val_loss: 0.6617 - val_acc: 0.7160
Epoch 3/15
414/414 [==============================] - 11s 26ms/step - loss: 0.6025 - acc: 0.7496 - val_loss: 0.5463 - val_acc: 0.7963
Epoch 4/15
414/414 [==============================] - 12s 29ms/step - loss: 0.5521 - acc: 0.7795 - val_loss: 0.6138 - val_acc: 0.7773
Epoch 5/15
414/414 [==============================] - 12s 30ms/step - loss: 0.5083 - acc: 0.7989 - val_loss: 0.5334 - val_acc: 0.7963
Epoch 6/15
414/414 [==============================] - 11s 26ms/step - loss: 0.4704 - acc: 0.8188 - val_loss: 0.5252 - val_acc: 0.7956
Epoch 7/15
414/414 [==============================] - 11s 26ms/step - loss: 0.4564 - acc: 0.8288 - val_loss: 0.4886 - val_acc: 0.8273
Epoch 8/15
414/414 [==============================] - 11s 26ms

In [102]:
from keras.models import load_model
model = load_model('/content/model_best_100.hdf5')

In [103]:
y_pred = np.argmax(model.predict(X_test), axis=1)
y_test_1 = np.argmax(y_test, axis=1)

In [104]:
from sklearn.metrics import classification_report

accu = classification_report(y_test_1, y_pred, output_dict = True)['accuracy']
history_LSTM.append(accu)

# save history

In [105]:
import pickle
with open('LSTM_history.pkl', 'wb') as f:
  pickle.dump(history_LSTM, f)

In [106]:
history_LSTM

[0.7054263565891473,
 0.7364341085271318,
 0.7653276955602537,
 0.7935165609584214,
 0.7857646229739254,
 0.8005637773079634,
 0.8054968287526427,
 0.806906272022551,
 0.8259337561663143,
 0.8273431994362227]